In [40]:
'''处理学姐给的resultxxxx.json数据'''

import os, json, re, hashlib
from tqdm import tqdm

def get_province(affiliations):
    provinces = ["河北","山西","黑龙江","吉林","辽宁","江苏","浙江","安徽","福建","江西","山东","河南","湖北","湖南","广东","海南","四川","贵州","云南","陕西","甘肃","青海","台湾","内蒙古","广西","西藏","宁夏","新疆","北京","天津","上海","重庆","香港","澳门"]

    cities = {
        "北京":[
            "北京"
        ],
        "上海":[
            "上海"
        ],
        "重庆":[
            "重庆","西南政法大学"
        ],
        "天津":[
            "天津"
        ],
        "河北":[
            "河北","石家庄","唐山","秦皇岛","邯郸","邢台","保定","张家口","承德","沧州","廊坊","衡水"
        ],
        "山西":[
            "山西","太原","大同","阳泉","长治","晋城","朔州","晋中","运城","忻州","临汾","吕梁"
        ],
        "内蒙古":[
            "内蒙古","呼和浩特","包头","乌海","赤峰","通辽","鄂尔多斯","呼伦贝尔","巴彦淖尔","乌兰察布","兴安盟","锡林郭勒盟","伊克昭盟","阿拉善"
        ],
        "辽宁":[
            "辽宁","沈阳","大连","鞍山","抚顺","本溪","丹东","锦州","营口","阜新","辽阳","盘锦","铁岭","朝阳","葫芦岛"
        ],
        "吉林":[
            "吉林","长春","吉林","四平","辽源","通化","白山","松原","白城","延边"
        ],
        "黑龙江":[
            "黑龙江","哈尔滨","齐齐哈尔","鸡西","鹤岗","双鸭山","大庆","伊春","佳木斯","七台河","牡丹江","黑河","绥化","大兴安岭"
        ],
        "江苏":[
            "江苏","南京","无锡","徐州","常州","苏州","南通","连云港","淮安","盐城","扬州","镇江","泰州","宿迁","淮阴"
        ],
        "浙江":[
            "浙江","杭州","宁波","温州","嘉兴","湖州","绍兴","金华","衢州","舟山","台州","丽水"
        ],
        "安徽":[
            "安徽","合肥","芜湖","蚌埠","淮南","马鞍山","淮北","铜陵","安庆","黄山","阜阳","宿州","滁州","六安","宣城","池州","亳州"
        ],
        "福建":[
            "福建","福州","厦门","莆田","三明","泉州","漳州","南平","龙岩","宁德"
        ],
        "江西":[
            "江西","南昌","景德镇","萍乡","九江","抚州","鹰潭","赣州","吉安","宜春","新余","上饶","赣南"
        ],
        "山东":[
            "山东","济南","青岛","淄博","枣庄","东营","烟台","潍坊","济宁","泰安","威海","日照","临沂","德州","聊城","滨州","菏泽"
        ],
        "河南":[
            "河南","郑州","开封","洛阳","平顶山","安阳","鹤壁","新乡","焦作","濮阳","许昌","漯河","三门峡","南阳","商丘","信阳","周口","驻马店","济源"
        ],
        "湖北":[
            "湖北","武汉","黄石","十堰","宜昌","襄阳","鄂州","荆门","孝感","荆州","黄冈","咸宁","随州","恩施","仙桃","潜江","天门","神农架"
        ],
        "湖南":[
            "湖南","长沙","株洲","湘潭","衡阳","邵阳","岳阳","常德","张家界","益阳","郴州","永州","怀化","娄底","湘西"
        ],
        "广东":[
            "广东","广州","韶关","深圳","珠海","汕头","佛山","江门","湛江","茂名","肇庆","惠州","梅州","汕尾","河源","阳江","清远","东莞","中山","潮州","揭阳","云浮"
        ],
        "广西":[
            "广西","南宁","柳州","桂林","梧州","北海","防城港","钦州","贵港","玉林","百色","贺州","河池","来宾","崇左"
        ],
        "海南":[
            "海南","海口","三亚","三沙","儋州","琼海","五指山","文昌","万宁","东方","定安","屯昌","澄迈","临高","白沙","昌江","乐东","陵水","保亭","琼中"
        ],
        "四川":[
            "四川","成都","自贡","攀枝花","泸州","德阳","绵阳","广元","遂宁","内江","乐山","南充","眉山","宜宾","广安","达州","雅安","巴中","资阳","阿坝","甘孜","凉山","巴中","眉山","资阳"
        ],
        "贵州":[
            "贵州","贵阳","六盘水","遵义","安顺","毕节","铜仁","黔西南","黔东南","安顺"
        ],
        "云南":[
            "云南","昆明","曲靖","玉溪","保山","昭通","丽江","普洱","临沧","楚雄","红河哈尼","文山","思茅","西双版纳","大理","德宏","怒江"
        ],
        "西藏":[
            "西藏","拉萨","日喀则","昌都","林芝","山南","那曲","阿里"
        ],
        "陕西":[
            "陕西","西安","铜川","宝鸡","咸阳","渭南","延安","汉中","榆林","安康","商洛"
        ],
        "甘肃":[
            "甘肃","兰州","嘉峪关","金昌","白银","天水","武威","张掖","平凉","酒泉","庆阳","定西","陇南","临夏","甘南"
        ],
        "青海":[
            "青海","西宁","海东","玉树","果洛","黄南","海西","海北","海南藏族"
        ],
        "宁夏":[
            "宁夏","银川","石嘴山","吴忠","固原","中卫"
        ],
        "新疆":[
            "新疆","乌鲁木齐","克拉玛依","吐鲁番","哈密","昌吉","博尔塔","巴音郭楞","阿克苏","克孜勒苏柯尔克孜","喀什","和田","伊犁","塔城","阿勒泰","石河子","阿拉尔","图木舒克","五家渠"
        ],
        "台湾":[
            "台湾","台北","高雄","基隆","台中","台南","新北","桃源","宜兰","新竹","嘉义"
        ],
        "香港":[
            "香港"
        ],
        "澳门":[
            "澳门"
        ],
    }

    paper_province = []
    for province in provinces:
        for city in cities[province]:
            if re.search(city, affiliations)!=None:
                paper_province.append(province)
                break
    return paper_province

def preprocess(entry):  # 将作者和单位转换为列表，有数字出现的打上标签authors[] affiliations[] tags[]
    entry['tags']=[]

    # 检查所有条目是否全面
    for value in entry.values():
        if value=="none" or value=='':
            entry['tags'].append("incomplete")

    # 处理作者
    delimiter_pattern=re.compile(r'[，；; \t\s]')
    authors = entry.get('authors')
    delimiters_author = delimiter_pattern.search(authors)
    # commas_authors=re.search(r'，',authors)
    # semicolons_authors=re.search(r'；',authors)
    numbers = re.findall(r'\d', authors)
    if numbers:  # 有数字
        entry['tags'].append('with nums')
    if authors=="none" or authors=='':
        entry['authors']=[]
        entry['tags'].append("authors missing")
    elif delimiters_author:
        entry['authors']=re.split(delimiter_pattern,authors)
    else:  # 只有一个作者
        entry['authors']=[authors]

    # 处理省份
    if type(entry['province'])!=list:
        province_list = []
        province_list = get_province(entry.get('affiliations'))
        if entry['province'] not in province_list:
            province_list.append(entry['province'])
        entry['province'] = province_list

    delimiter_pattern=re.compile(r'[,，；; \t\s]')
    # 处理单位
    affiliations = entry.get('affiliations')
    delimiters_affiliation = delimiter_pattern.search(affiliations)
    if affiliations=="none" or affiliations=='':
        entry['affiliations']=[]
        entry['tags'].append("keywords missing")
    elif delimiters_affiliation:
        entry['affiliations']=re.split(delimiter_pattern,affiliations)
    else:  # 只有一个单位
        entry['affiliations'] = [affiliations]

    # 处理关键词
    keywords = entry.get('keywords')
    keywords.strip()
    delimiters_affiliation = delimiter_pattern.search(keywords)
    if keywords=="none" or keywords=='':
        entry['keywords']=[]
        entry['tags'].append("keywords missing")
    elif delimiters_affiliation:
        tmp_keywords = re.split(delimiter_pattern,keywords)
        entry['keywords']=[]
        for tmp_keyword in tmp_keywords:
            if tmp_keyword!='':
                entry['keywords'].append(tmp_keyword)
    else:  # 只有一个单位
        entry['keywords'] = [keywords]

    # 处理引用
    citation = entry.get('citation')
    if type(citation)==str:
        citation.strip()
        try:
            citation = int(citation)
        except:
            entry['tags'].append("citation error")
    entry['citation'] = citation

    # 处理年份
    year = entry.get('year')
    if type(year)==str:
        year.strip()
        try:
            year = int(year)
        except:
            entry['tags'].append("year error")
    entry['year'] = year

    # 生成UID
    metadata_str=f"{entry['title']}{entry['source']}"
    # 将中文字符转换为字节序列（UTF-8 编码）
    metadata_bytes=metadata_str.encode('utf-8')
    uid=hashlib.sha256(metadata_bytes).hexdigest()
    entry['uid']=uid

    return entry

result_json_path = r"D:\vscode_workspace\ZhongYiPapers\database\分省份数据"
final_json_save_path = r"D:\vscode_workspace\ZhongYiPapers\database\final\tmp.json"
filenames = os.listdir(result_json_path)
result_json_list = []
for filename in filenames:
    with open(result_json_path+r"/"+filename, 'r', encoding='utf-8') as f:
        items_json_list = json.load(f)
        result_json_list.extend(items_json_list)
total_num = len(result_json_list)
# total_num = 10
pbar = tqdm(range(total_num))
final_item_dict = {}
for item_index in pbar:
    if 'title' not in result_json_list[item_index]:
        continue
    tmp = {}
    pbar.set_description("Processing %s" % item_index + "/%s"%total_num) # 设置描述
    tmp = preprocess(result_json_list[item_index])
    final_item_dict[tmp['uid']] = tmp

# utf-8将一个汉字编码为3个字节,gbk将一个汉字编码为2个字节,
with open(final_json_save_path, 'w', encoding='utf-8') as new_f:  # 重新写入
    json.dump(final_item_dict, new_f, ensure_ascii=False, indent=4) # 最后一个参数会保证dump之后的结果所有的字符都能被ascii表示




Processing 411362/411363: 100%|██████████| 411363/411363 [05:47<00:00, 1184.55it/s]


In [38]:
'''处理学姐给的resultxxxx.json数据'''

import os, json, re, hashlib
from tqdm import tqdm

def preprocess(entry):  # 将作者和单位转换为列表，有数字出现的打上标签authors[] affiliations[] tags[]
    entry['tags']=[]

    # 检查所有条目是否全面
    for value in entry.values():
        if value=="none" or value=='':
            entry['tags'].append("incomplete")

    # 处理作者
    delimiter_pattern=re.compile(r'[，；; \t\s]')
    authors = entry.get('authors')
    delimiters_author = delimiter_pattern.search(authors)
    # commas_authors=re.search(r'，',authors)
    # semicolons_authors=re.search(r'；',authors)
    numbers = re.findall(r'\d', authors)
    if numbers:  # 有数字
        entry['tags'].append('with nums')
    if authors=="none" or authors=='':
        entry['authors']=[]
        entry['tags'].append("authors missing")
    elif delimiters_author:
        entry['authors']=re.split(delimiter_pattern,authors)
    else:  # 只有一个作者
        entry['authors']=[authors]

    delimiter_pattern=re.compile(r'[,，；; \t\s]')
    # 处理单位
    affiliations = entry.get('affiliations')
    delimiters_affiliation = delimiter_pattern.search(affiliations)
    if affiliations=="none" or affiliations=='':
        entry['affiliations']=[]
        entry['tags'].append("keywords missing")
    elif delimiters_affiliation:
        entry['affiliations']=re.split(delimiter_pattern,affiliations)
    else:  # 只有一个单位
        entry['affiliations'] = [affiliations]

    # 处理关键词
    keywords = entry.get('keywords')
    keywords.strip()
    delimiters_affiliation = delimiter_pattern.search(keywords)
    if keywords=="none" or keywords=='':
        entry['keywords']=[]
        entry['tags'].append("keywords missing")
    elif delimiters_affiliation:
        tmp_keywords = re.split(delimiter_pattern,keywords)
        entry['keywords']=[]
        for tmp_keyword in tmp_keywords:
            if tmp_keyword!='':
                entry['keywords'].append(tmp_keyword)
    else:  # 只有一个单位
        entry['keywords'] = [keywords]

    # 处理省份
    if type(entry['province'])!=list:
        entry['province'] = [entry['province']]

    # 处理引用
    citation = entry.get('citation')
    if type(citation)==str:
        citation.strip()
        try:
            citation = int(citation)
        except:
            entry['tags'].append("citation error")
    entry['citation'] = citation

    # 处理年份
    year = entry.get('year')
    if type(year)==str:
        year.strip()
        try:
            year = int(year)
        except:
            entry['tags'].append("year error")
    entry['year'] = year

    # 生成UID
    metadata_str=f"{entry['title']}{entry['source']}"
    # 将中文字符转换为字节序列（UTF-8 编码）
    metadata_bytes=metadata_str.encode('utf-8')
    uid=hashlib.sha256(metadata_bytes).hexdigest()
    entry['uid']=uid

    return entry

result_json_path = r"D:\vscode_workspace\ZhongYiPapers\database\lzd数据\papers_unnumed_authers.json"
final_json_save_path = r"D:\vscode_workspace\ZhongYiPapers\database\final\tmp.json"
result_json_list = []
with open(result_json_path, encoding='utf-8') as f:
    result_json_list = json.load(f)
total_num = len(result_json_list)
# total_num = 10
pbar = tqdm(range(total_num))
final_item_dict = {}
for item_index in pbar:
    if 'title' not in result_json_list[item_index]:
        continue
    tmp = {}
    pbar.set_description("Processing %s" % item_index + "/%s"%total_num) # 设置描述
    tmp = preprocess(result_json_list[item_index])
    final_item_dict[tmp['uid']] = tmp

# utf-8将一个汉字编码为3个字节,gbk将一个汉字编码为2个字节,
with open(final_json_save_path, 'w', encoding='utf-8') as new_f:  # 重新写入
    json.dump(final_item_dict, new_f, ensure_ascii=False, indent=4) # 最后一个参数会保证dump之后的结果所有的字符都能被ascii表示




Processing 18857/18858: 100%|██████████| 18858/18858 [00:09<00:00, 2067.24it/s]


In [60]:
with open(r"D:\vscode_workspace\ZhongYiPapers\database\final\lzd_numed.json", encoding='utf-8') as f:
    lzd_numed = json.load(f)
    lzd_numed_keys = list(lzd_numed.keys())
with open(r"D:\vscode_workspace\ZhongYiPapers\database\final\lzd_unnumed.json", encoding='utf-8') as f:
    lzd_unnumed = json.load(f)
    lzd_unnumed_keys = list(lzd_unnumed.keys())
with open(r"D:\vscode_workspace\ZhongYiPapers\database\final\tmp.json", encoding='utf-8') as f:
    fby = json.load(f)
    fby_keys = list(fby.keys())

final_uids = []
final_jsons_dict = {}

for lzd_numed_key in lzd_numed_keys:
    final_uids.append(lzd_numed_key)
    tmp = {}
    tmp = lzd_numed[lzd_numed_key]
    tmp['tags'].append('lzd')
    final_jsons_dict[lzd_numed_key] = tmp

for lzd_unnumed_key in lzd_unnumed_keys:
    final_uids.append(lzd_unnumed_key)
    tmp = {}
    tmp = lzd_unnumed[lzd_unnumed_key]
    tmp['tags'].append('lzd')
    final_jsons_dict[lzd_unnumed_key] = tmp

for fby_key in fby_keys:
    final_uids.append(fby_key)
    tmp = {}
    tmp = fby[fby_key]
    tmp['tags'].append('fby')
    final_jsons_dict[fby_key] = tmp

final_uids.sort()
final = [final_uids, final_jsons_dict]

with open(r"D:\vscode_workspace\ZhongYiPapers\database\final\final.json", 'w', encoding='utf-8') as new_f:  # 重新写入
    json.dump(final, new_f, ensure_ascii=False, indent=4) # 最后一个参数会保证dump之后的结果所有的字符都能被ascii表示

In [61]:
a = {
    "tags":[777]
}
b = {}
b = a
tmp_tag = b.get('tags')
b['tags'].append(666)
b = [123,456]
c = [a, b]
c

[{'tags': [777, 666]}, [123, 456]]

In [59]:
a = ["1sad5a6sf", "asdf87as98s8d", "7as8", "sdaf48dx9q"]
a.sort()
a

['1sad5a6sf', '7as8', 'asdf87as98s8d', 'sdaf48dx9q']